In [9]:
from align.spark.schemas import ARCHIVE_ORG_SCHEMA
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('CC-BY-license').getOrCreate()
input_catalogue_path = "gs://the-peoples-speech-west-europe/archive_org/Mar_7_2021/EXPANDED_LICENSES_FILTERED_ACCESS.jsonl.gz"
df = spark.read.format('json').schema(ARCHIVE_ORG_SCHEMA).load(input_catalogue_path)
print(df.count())
df.show()

63627
+----------+--------------------+--------------------+--------------------+--------------------+-----------+--------------------+-----------------+-----------+--------------------+
|   created|                  d1|                  d2|                 dir|               files|files_count|          identifier|item_last_updated|  item_size|            metadata|
+----------+--------------------+--------------------+--------------------+--------------------+-----------+--------------------+-----------------+-----------+--------------------+
|1615341644|ia600307.us.archi...|ia800307.us.archi...|/28/items/0084_To...|[{null, null, f65...|         44|0084_Tomorrow_Alw...|       1561422845| 1065371528|{null, 2012-08-25...|
|1615341644|ia902800.us.archi...|ia802800.us.archi...|/5/items/00Bienve...|[{null, null, 35a...|         21|00BienvenidoALaRe...|       1613076782|   48528872|{null, 2018-09-18...|
|1615341644|ia902806.us.archi...|ia802806.us.archi...|/13/items/00Bienv...|[{null, null, 

In [10]:
columns = [df.metadata.licenseurl, df.metadata.creator, df.metadata.title, df.metadata.credits]
licenses = ['https://creativecommons.org/licenses/by/4.0/',
'https://creativecommons.org/licenses/by/3.0/',
'http://creativecommons.org/licenses/by/2.0/',
'http://creativecommons.org/licenses/by/3.0/',
'https://creativecommons.org/licenses/by/3.0',
'http://creativecommons.org/licenses/by/2.5/',
'http://creativecommons.org/licenses/by/2.5/pl/',
'http://creativecommons.org/licenses/by/2.5/it/',
'http://creativecommons.org/licenses/by/4.0/',
'http://creativecommons.org/licenses/by/3.0/us/',
'http://creativecommons.org/licenses/by/2.5/ar/']
df = df.select(columns)
df.show()

+--------------------+--------------------+--------------------+--------------------+
| metadata.licenseurl|    metadata.creator|      metadata.title|    metadata.credits|
+--------------------+--------------------+--------------------+--------------------+
|http://creativeco...|Lamont-Clemens, Inc.|Tomorrow Always C...|Produced by Burne...|
|http://creativeco...|                AT86|00 Bienvenido A L...|                null|
|http://creativeco...|          Matrixland|00 Bienvenido A L...|                null|
|http://creativeco...|                Open|Open Friday | Apr...|                null|
|http://creativeco...|                Open|02777 A Open Apri...|                null|
|http://creativeco...|  Francisco Betances|Open Monday | Jun...|                null|
|http://creativeco...|  Francisco Betances|Open Wednesday | ...|                null|
|http://creativeco...|  Francisco Betances|Open Wednesday | ...|                null|
|http://creativeco...|                Open|Open Wednes

In [11]:
c = '`metadata.licenseurl`'
df.agg(F.count(F.when(F.isnull(c), c)).alias('NULL_Count')).show()

+----------+
|NULL_Count|
+----------+
|         4|
+----------+



### Look NA values

In [12]:
df_pd = df.toPandas()
df_pd.head()

,metadata.licenseurl,metadata.creator,metadata.title,metadata.credits
0,http://creativecommons.org/publicdomain/mark/1.0/,"Lamont-Clemens, Inc.",Tomorrow Always Comes,Produced by Burnet Lamont; Directed by William...
1,http://creativecommons.org/publicdomain/mark/1.0/,AT86,00 Bienvenido A La Revolución De La Impresión ...,None
2,http://creativecommons.org/publicdomain/mark/1.0/,Matrixland,00 Bienvenido A La Revolución De La Impresión ...,None
3,http://creativecommons.org/publicdomain/mark/1.0/,Open,"Open Friday | Apr. 12, 2013",None
4,http://creativecommons.org/publicdomain/mark/1.0/,Open,02777 A Open April 17 13,None


In [13]:
df_pd.shape

(63627, 4)

In [14]:
df_pd.isna().sum()

metadata.licenseurl        4
metadata.creator       48250
metadata.title             4
metadata.credits       61509
dtype: int64

In [15]:
df_pd['metadata.licenseurl'].value_counts(normalize=True) * 100

https://www.usa.gov/government-works                                                                                                                                                             73.350832
http://creativecommons.org/licenses/publicdomain/                                                                                                                                                 8.393191
http://creativecommons.org/publicdomain/mark/1.0/                                                                                                                                                 6.626534
https://creativecommons.org/licenses/by/3.0/                                                                                                                                                      6.112569
http://creativecommons.org/publicdomain/zero/1.0/                                                                                                                                           

In [16]:
df_pd[df_pd['metadata.licenseurl'].isna()]

,metadata.licenseurl,metadata.creator,metadata.title,metadata.credits
7305,None,None,None,None
7702,None,None,None,None
7703,None,None,None,None
7711,None,None,None,None


In [17]:
df_pd['metadata.licenseurl'].unique()

array(['http://creativecommons.org/publicdomain/mark/1.0/',
       'https://creativecommons.org/licenses/by/3.0/',
       'http://creativecommons.org/publicdomain/zero/1.0/',
       'http://creativecommons.org/licenses/by/3.0/',
       'https://creativecommons.org/licenses/by/3.0',
       'http://creativecommons.org/licenses/publicdomain/',
       'http://creativecommons.org/licenses/by/4.0/',
       'https://creativecommons.org/licenses/cc0/3.0/',
       'https://creativecommons.org/licenses/by/4.0/',
       'http://creativecommons.org/licenses/by/3.0/us/',
       'http://creativecommons.org/licenses/by/2.5/',
       'http://creativecommons.org/licenses/by/2.0/',
       'http://creativecommons.org/licenses/by/2.5/pl/',
       'http://creativecommons.org/licenses/by/2.5/it/', None,
       '["https://creativecommons.org/licenses/by/3.0/","https://creativecommons.org/licenses/by/3.0/","https://creativecommons.org/licenses/by/3.0/","https://creativecommons.org/licenses/by/3.0/"]',
       

## Filter values by  license

In [18]:
df = (df.withColumnRenamed('metadata.licenseurl','licenseurl').withColumnRenamed('metadata.creator', 'creator')
     .withColumnRenamed('metadata.title', 'title').withColumnRenamed('metadata.credits', 'credits'))
df = df.dropna(subset=['licenseurl'])
df.show()

+--------------------+--------------------+--------------------+--------------------+
|          licenseurl|             creator|               title|             credits|
+--------------------+--------------------+--------------------+--------------------+
|http://creativeco...|Lamont-Clemens, Inc.|Tomorrow Always C...|Produced by Burne...|
|http://creativeco...|                AT86|00 Bienvenido A L...|                null|
|http://creativeco...|          Matrixland|00 Bienvenido A L...|                null|
|http://creativeco...|                Open|Open Friday | Apr...|                null|
|http://creativeco...|                Open|02777 A Open Apri...|                null|
|http://creativeco...|  Francisco Betances|Open Monday | Jun...|                null|
|http://creativeco...|  Francisco Betances|Open Wednesday | ...|                null|
|http://creativeco...|  Francisco Betances|Open Wednesday | ...|                null|
|http://creativeco...|                Open|Open Wednes

In [21]:
regexp = r"(http|https)://creativecommons.org/licenses/by/(1[.]0|2[.]0|2[.]5|3[.]0|4[.]0)"
df = df.filter(df['licenseurl'].rlike(regexp))
df.show()

+--------------------+--------------------+--------------------+-------+
|          licenseurl|             creator|               title|credits|
+--------------------+--------------------+--------------------+-------+
|https://creativec...|     City of Holland|1-12-16 Early Bir...|   null|
|https://creativec...|     City of Holland|1-12-16 Planning ...|   null|
|https://creativec...|     City of Holland|1-18-17 City Council|   null|
|https://creativec...|     City of Holland|1-18-18 City Coun...|   null|
|https://creativec...|     City of Holland|1-21 City Council...|   null|
|https://creativec...|     City of Holland|1-22 City Council...|   null|
|https://creativec...|     City of Holland|1-23 Board of App...|   null|
|https://creativec...|     City of Holland|1-25-18 Board of ...|   null|
|https://creativec...|     City of Holland|1-26-17 Board of ...|   null|
|https://creativec...|     City of Holland|1-28-16 Board of ...|   null|
|https://creativec...|     City of Holland|1-3-18 C

In [23]:
len(df.columns)

4

## Types of licences

In [20]:
licenses_url = df.select('licenseurl').distinct().collect()
licenses_urls = []
for license in licenses_url:
    licenses_urls.append(license[0])
licenses_urls

['https://creativecommons.org/licenses/by/4.0/',
 '["https://creativecommons.org/licenses/by/3.0/","https://creativecommons.org/licenses/by/3.0/","https://creativecommons.org/licenses/by/3.0/","https://creativecommons.org/licenses/by/3.0/"]',
 'https://creativecommons.org/licenses/by/3.0/',
 'http://creativecommons.org/licenses/by/2.0/',
 'http://creativecommons.org/licenses/by/3.0/',
 'https://creativecommons.org/licenses/by/3.0',
 'http://creativecommons.org/licenses/by/2.5/',
 'http://creativecommons.org/licenses/by/2.5/pl/',
 'http://creativecommons.org/licenses/by/2.5/it/',
 'http://creativecommons.org/licenses/by/4.0/',
 'http://creativecommons.org/licenses/by/3.0/us/',
 'http://creativecommons.org/licenses/by/2.5/ar/']

## Convert to SQL

In [12]:
df.createOrReplaceTempView("CC_BY_creators")

In [13]:
spark.sql('SELECT * FROM CC_BY_creators').show()

+--------------------+--------------------+--------------------+-------+
|          licenseurl|             creator|               title|credits|
+--------------------+--------------------+--------------------+-------+
|https://creativec...|     City of Holland|1-12-16 Early Bir...|   null|
|https://creativec...|     City of Holland|1-12-16 Planning ...|   null|
|https://creativec...|     City of Holland|1-18-17 City Council|   null|
|https://creativec...|     City of Holland|1-18-18 City Coun...|   null|
|https://creativec...|     City of Holland|1-21 City Council...|   null|
|https://creativec...|     City of Holland|1-22 City Council...|   null|
|https://creativec...|     City of Holland|1-23 Board of App...|   null|
|https://creativec...|     City of Holland|1-25-18 Board of ...|   null|
|https://creativec...|     City of Holland|1-26-17 Board of ...|   null|
|https://creativec...|     City of Holland|1-28-16 Board of ...|   null|
|https://creativec...|     City of Holland|1-3-18 C

MapPartitionsRDD[43] at toJavaRDD at NativeMethodAccessorImpl.java:0

## Save as txt

In [ ]:
def myConcat(*cols):
    concat_columns = []
    for c in cols[:-1]:
        concat_columns.append(F.coalesce(c, F.lit("*")))
        concat_columns.append(F.lit(" "))  
    concat_columns.append(F.coalesce(cols[-1], F.lit("*")))
    return F.concat(*concat_columns)

df = df.withColumn("credits", myConcat(*df.columns)).select("credits")
df.coalesce(1).write.format("text").option("header", "false").mode("append").save("credits.txt")